<a href="https://colab.research.google.com/github/michaelwnau/ai_academy_notebooks/blob/main/image_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import tensorflow as tf

In [4]:
# Prompt user for image file path
image_path = input("Enter the file path to the image file: ")

Enter the file path to the image file: /content/drive/MyDrive/zebramusselsOne.jpg


In [5]:
# Prompt user for GPU usage
use_gpu = input("Will you use a local GPU? (yes/no): ")
if use_gpu.lower() == "yes":
    # Enable CUDA for GPU usage
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    physical_devices = tf.config.experimental.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Will you use a local GPU? (yes/no): no


In [6]:
# Prompt user for TPU usage
use_tpu = input("Will you use a local TPU? (yes/no): ")
if use_tpu.lower() == "yes":
    # Activate Coral library for TPU usage
    from tflite_runtime.interpreter import load_delegate
    from tflite_runtime.interpreter import Interpreter

    interpreter = Interpreter(model_path="path/to/edgetpu.tflite",
                              experimental_delegates=[load_delegate('libedgetpu.so.1.0')])

Will you use a local TPU? (yes/no): no


In [7]:
# Load the image
image = tf.io.read_file(image_path)
image = tf.image.decode_image(image, channels=3)
image = tf.image.resize(image, (224, 224))  # Adjust the desired size


In [8]:
# Perform image detection using a pre-trained model
if use_tpu.lower() == "yes":
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    interpreter.allocate_tensors()
    interpreter.set_tensor(input_details[0]['index'], image[tf.newaxis, ...])
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index'])
else:
    # Use a different model or method for GPU or CPU inference
    # Replace the code below with your specific implementation
    model = tf.keras.applications.MobileNetV2(weights='imagenet')
    image = tf.expand_dims(image, axis=0)
    predictions = model.predict(image)


1/1 [==============================] - 1s 995ms/step


In [9]:
# Print the results
print("Predictions:")
for idx, pred in enumerate(predictions[0]):
    print(f"Class {idx}: {pred}")


Predictions:
Class 0: 2.7458410841063596e-05
Class 1: 7.72572893765755e-05
Class 2: 3.0490612061839784e-06
Class 3: 6.250780643313192e-06
Class 4: 4.2471718188608065e-05
Class 5: 5.827930181112606e-06
Class 6: 5.0615082727745175e-05
Class 7: 5.673636678693583e-06
Class 8: 2.856161245290423e-06
Class 9: 2.2814352632849477e-05
Class 10: 1.0640767868608236e-05
Class 11: 2.9751224701612955e-06
Class 12: 2.8289889087318443e-05
Class 13: 2.1276797269820236e-05
Class 14: 2.2259435354499146e-05
Class 15: 5.237177447270369e-06
Class 16: 2.2264934159466065e-05
Class 17: 9.243351996701676e-06
Class 18: 1.1367706065357197e-05
Class 19: 4.962341336067766e-05
Class 20: 3.4397729905322194e-05
Class 21: 1.264181628357619e-05
Class 22: 1.0283126357535366e-05
Class 23: 1.321766831097193e-05
Class 24: 2.8048703825334087e-05
Class 25: 3.5327811929164454e-05
Class 26: 3.78667791665066e-05
Class 27: 1.872874236141797e-05
Class 28: 1.9631597751867957e-05
Class 29: 7.838754754629917e-06
Class 30: 1.5258994608

In [2]:
import os
import torch
from torchvision import transforms
from PIL import Image

# Prompt user for image file path
image_path = input("Enter the file path to the image file: ")

# Prompt user for GPU usage
use_gpu = input("Will you use a local GPU? (yes/no): ")
if use_gpu.lower() == "yes":
    # Check if CUDA is available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch.backends.cudnn.benchmark = True
    else:
        print("GPU is not available. Using CPU instead.")
        device = torch.device("cpu")
else:
    device = torch.device("cpu")

# Prompt user for TPU usage
use_tpu = input("Will you use a local TPU? (yes/no): ")
if use_tpu.lower() == "yes":
    # Activate Coral library for TPU usage
    from torchvision.models import mobilenet_v2
    from torch.utils.mobile_optimizer import optimize_for_mobile

    model = mobilenet_v2(pretrained=False)
    model.load_state_dict(torch.load("path/to/edgetpu.pth"))
    model = optimize_for_mobile(model)
    model = model.to(device)
    model.eval()
else:
    # Use a different model or method for GPU or CPU inference
    # Replace the code below with your specific implementation
    model = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)
    model = model.to(device)
    model.eval()

# Load and preprocess the image
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
image = Image.open(image_path)
image = preprocess(image)
image = image.unsqueeze(0).to(device)

# Perform image detection using the pre-trained model
with torch.no_grad():
    predictions = model(image)

# Print the results
print("Predictions:")
_, indices = torch.sort(predictions, descending=True)
for idx in indices[0]:
    print(f"Class {idx.item()}: {predictions[0][idx].item()}")



Enter the file path to the image file: /content/drive/MyDrive/zebramusselsOne.jpg
Will you use a local GPU? (yes/no): no
Will you use a local TPU? (yes/no): no


Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 112MB/s] 


Predictions:
Class 794: 9.860587120056152
Class 619: 6.338293075561523
Class 783: 6.230910301208496
Class 488: 6.203638076782227
Class 549: 5.677371025085449
Class 903: 5.6077799797058105
Class 599: 5.507816314697266
Class 700: 5.407959938049316
Class 584: 5.398898124694824
Class 434: 5.391544342041016
Class 793: 5.289090633392334
Class 772: 5.149414539337158
Class 883: 5.0641703605651855
Class 578: 5.0600481033325195
Class 314: 5.036128997802734
Class 446: 4.910945892333984
Class 894: 4.835638046264648
Class 905: 4.810186386108398
Class 399: 4.77323055267334
Class 470: 4.731071949005127
Class 512: 4.712701797485352
Class 922: 4.683281421661377
Class 788: 4.636566162109375
Class 824: 4.632028579711914
Class 892: 4.580539703369141
Class 911: 4.507868766784668
Class 552: 4.502068042755127
Class 473: 4.50131893157959
Class 310: 4.478097438812256
Class 868: 4.477102756500244
Class 591: 4.439397811889648
Class 600: 4.407419681549072
Class 789: 4.368145942687988
Class 515: 4.356588363647461
